In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

In [15]:
## Load the trained model, scaler pickle file, onehot
model = load_model('model.h5')

## Load the encoders and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encode_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [16]:
## Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 30,
    'Tenure': 2,
    'Balance': 1000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [17]:
## One Hot encode 'Geography'
geo_encoded = label_encode_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Piyush.Bhatnagar\Videos\Practical Implemetation\ANN Practical implementation\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [25]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,30,2,1000,2,1,1,50000


In [27]:
## Encode categorical Variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,30,2,1000,2,1,1,50000


In [30]:
## Concatenation one hot encoded data
# Concatenate one-hot encoded data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,30,2,1000,2,1,1,50000,1.0,0.0,0.0


In [31]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755, -0.84593077, -1.04241787, -1.20245957,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [32]:
## Predict Churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


array([[0.01062559]], dtype=float32)

In [33]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.010625594)

In [34]:
if prediction_proba > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is unlikely to churn")

The customer is unlikely to churn
